In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [53]:
p18 = pd.read_csv('players_18.csv', encoding='latin-1') #players_18.csv file, some of the cases we encoded in latin1
p19 = pd.read_csv('players_19.csv', encoding='latin-1') #players_19.csv file, some of the cases we encoded in latin1
m_results = pd.read_csv('match_result.csv', encoding='latin-1') #match_results.csv file, some of the cases we encoded in latin1

In [54]:
p18.shape

(17592, 17)

In [55]:
p19.shape

(17770, 17)

In [56]:
m_results.shape

(2984, 6)

In [57]:
#dropping unneeded variables from the player_18 and player_19 csv files first

#list of variables of interest (voi) that we want to keep for our models
voi_player = ['short_name', 'age', 'height_cm', 'weight_kg', 'club', 'wage_eur',
      'pace' ,'shooting', 'passing', 'dribbling', 'defending',
      'gk_handling', 'gk_speed', 'gk_kicking', 'gk_diving', 'gk_positioning']

columns_list_18 = p18.columns.values.tolist() #list of all columns
columns_list_19 = p19.columns.values.tolist()

for i in range(len(columns_list_18)): #iterate through list of columns
    if columns_list_18[i] not in voi_player: #if column is not in our variables of interest list
        p18.pop(columns_list_18[i]) #remove it from the df
        
for i in range(len(columns_list_19)):
    if columns_list_19[i] not in voi_player:
        p19.pop(columns_list_19[i])

In [58]:
p18['short_name'] = p18['short_name'].astype('category') #setting to correct variable types
p18['club'] = p18['club'].astype('category') #setting to correct variable types
p19['short_name'] = p19['short_name'].astype('category') #setting to correct variable types
p19['club'] = p19['club'].astype('category') #setting to correct variable types

In [59]:
p18.shape #confirming we cut out the 87 unneeded columns

(17592, 16)

In [60]:
p19.shape #confirming we cut out the 87 unneeded columns

(17770, 16)

In [61]:
#dropping unneeded variables from the match_results csv

#list of variables of interest (voi) that we want to keep for our models
voi_match = ['FTHG', 'FTAG', 'FTR', 'HomeTeam', 'AwayTeam']

columns_list_match = m_results.columns.values.tolist()

for i in range(len(columns_list_match)): #iterate through list of columns
    if columns_list_match[i] not in voi_match: #if column is not in our variables of interest list
        m_results.pop(columns_list_match[i]) #remove it from the df

In [62]:
m_results.shape

(2984, 5)

In [63]:
m_results.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Standard de Liège,KAA Gent,3,2,H
1,KV Kortrijk,RSC Anderlecht,1,4,A
2,KV Oostende,Royal Excel Mouscron,2,1,H
3,Sint-Truidense VV,KSV Cercle Brugge,0,0,D
4,SV Zulte-Waregem,Waasland-Beveren,2,2,D


In [64]:
m_results['HomeTeam'] = m_results['HomeTeam'].astype('category') #setting to correct variable types
m_results['AwayTeam'] = m_results['AwayTeam'].astype('category')
m_results['FTR'] = m_results['FTR'].astype('category')

In [65]:
# p18.to_csv('players_18.csv')
# p19.to_csv('players_19.csv')
# m_results.to_csv('match_result.csv')

In [66]:
#group by teams and calculate the standard deviation
p18_team_sd = p18.groupby('club').mean().sort_values('wage_eur',ascending=False)
p19_team_sd = p19.groupby('club').mean().sort_values('wage_eur',ascending=False)

In [67]:
#group by team and calculate the mean
p18_team_mean = p18.groupby('club').mean().sort_values('pace',ascending=False)
p19_team_mean = p19.groupby('club').mean().sort_values('pace',ascending=False)

In [68]:
#preparing to merge table, by standard deviation
HomeStatsSd = p18_team_sd
HomeStatsSd = HomeStatsSd.add_prefix('Home ')
HomeStatsSd = HomeStatsSd.reset_index()
AwayStatsSd = p18_team_sd
AwayStatsSd = AwayStatsSd.add_prefix('Away ')
AwayStatsSd = AwayStatsSd.reset_index()

In [69]:
temp = pd.merge(m_results,HomeStatsSd,'left',left_on='HomeTeam',right_on='club')
sd_table = pd.merge(temp, AwayStatsSd, 'left',left_on='AwayTeam',right_on='club')  #final table, all stats are sd

In [71]:
#preparing to merge table, by mean
HomeStatsMean = p18_team_mean
HomeStatsMean = HomeStatsMean.add_prefix('Home ')
HomeStatsMean = HomeStatsMean.reset_index()
AwayStatsMean = p18_team_mean
AwayStatsMean = AwayStatsMean.add_prefix('Away ')
AwayStatsMean = AwayStatsMean.reset_index()

In [72]:
temp = pd.merge(m_results,HomeStatsMean,'left',left_on='HomeTeam',right_on='club')
mean_table = pd.merge(temp, AwayStatsMean, 'left',left_on='AwayTeam',right_on='club')  #final table, all stats are mean